In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os.path, sys, re
import time
from PIL import Image

In [2]:
# cd ~/Desktop/Data_Science/Erdos_Institute/ecg-proj/ecg-copy/

In [3]:
pathroot = "new dataset"

In [4]:
"""
Note:
-----
1) 2)The value for the key 'HB' is a list of dictionaries 
corresponding to the cases
    i) HB <= 399
    ii) HB >= 400 HB <= 483
    iii) HB >= 484 HB <= 548 // only 12 leads ignored for now
    iv) HB >= 549

2)The value for the key 'MI' is a list of dictionaries 
corresponding to the cases
    i) MI <=37
    ii) MI >=38 MI <= 77
    iii) MI >= 78

3) The value of the key 'Covid' is broken down as follows:
    i) Binder1_Page file name
    ii) COVID file name

4) The value of the key 'PMI' corresponds to:
    i) PMI<113
    ii) 113<=PMI<=161 --- ATTENTION: only 12 Leads in the case! ignored for now
    iii) PMI>=162 PMI <= 203
    iv) PMI>=204
"""
superDict = {'Normal': {'Lead1': (130, 300, 640, 600), 'Lead2': (641, 300, 1125, 600), 'Lead3': (1130, 300, 1625, 600), 'Lead4': (1630, 300, 2120, 600),
                        'Lead5': (130, 600, 640, 900), 'Lead6': (641, 600, 1125, 900), 'Lead7': (1130, 600, 1625, 900), 'Lead8': (1630, 600, 2120, 900),
                        'Lead9': (130, 900, 640, 1200), 'Lead10': (641, 900, 1125, 1200), 'Lead11': (1130, 900, 1625, 1200), 'Lead12': (1630, 900, 2120, 1200),
                        'Lead13': (130, 1205, 2120, 1450)
                        },
             'HB': [{'Lead1': (130, 300, 640, 600), 'Lead2': (641, 300, 1125, 600), 'Lead3': (1130, 300, 1625, 600), 'Lead4': (1630, 300, 2110, 600),
                     'Lead5': (130, 600, 640, 900), 'Lead6': (641, 600, 1125, 900), 'Lead7': (1130, 600, 1625, 900), 'Lead8': (1630, 600, 2110, 900),
                     'Lead9': (130, 900, 640, 1200), 'Lead10': (641, 900, 1125, 1200), 'Lead11': (1130, 900, 1625, 1200), 'Lead12': (1630, 900, 2110, 1200),
                     'Lead13': (130, 1205, 2110, 1450)
                     },

                    {'Lead1': (125, 300, 1125, 500), 'Lead2': (1125, 300, 2125, 500),
                     'Lead3': (125, 500, 1125, 680), 'Lead4': (1125, 500, 2125, 680),
                     'Lead5': (125, 680, 1125, 845), 'Lead6': (1125, 680, 2125, 845),
                     'Lead7': (125, 845, 1125, 1000), 'Lead8': (1125, 845, 2125, 1000),
                     'Lead9': (125, 1000, 1125, 1150), 'Lead10': (1125, 1000, 2125, 1150),
                     'Lead11': (125, 1150, 1125, 1300), 'Lead12': (1125, 1150, 2125, 1300),
                     'Lead13': (125, 1300, 2125, 1490)},

                    {'Lead1': (130, 300, 640, 600), 'Lead2': (641, 300, 1125, 600), 'Lead3': (1130, 300, 1625, 600), 'Lead4': (1630, 300, 2110, 600),
                     'Lead5': (130, 600, 640, 900), 'Lead6': (641, 600, 1125, 900), 'Lead7': (1130, 600, 1625, 900), 'Lead8': (1630, 600, 2110, 900),
                     'Lead9': (130, 900, 640, 1200), 'Lead10': (641, 900, 1125, 1200), 'Lead11': (1130, 900, 1625, 1200), 'Lead12': (1630, 900, 2110, 1200),
                     'Lead13': (130, 1205, 2110, 1450)
                     }],

             'MI': [{'Lead1': (125, 310, 625, 600), 'Lead2': (640, 310, 1125, 600), 'Lead3': (1140, 310, 1625, 600), 'Lead4': (1640, 310, 2125, 600),
                     'Lead5': (125, 605, 625, 900), 'Lead6': (640, 605, 1125, 900), 'Lead7': (1140, 605, 1625, 900), 'Lead8': (1640, 605, 2125, 900),
                     'Lead9': (125, 905, 625, 1200), 'Lead10': (640, 905, 1125, 1200), 'Lead11': (1140, 905, 1625, 1200), 'Lead12': (1640, 905, 2125, 1200),
                     'Lead13': (125, 1205, 2125, 1495)},

                    {'Lead1': (125, 300, 1125, 500), 'Lead2': (1125, 300, 2125, 500),
                     'Lead3': (125, 500, 1125, 680), 'Lead4': (1125, 500, 2125, 680),
                     'Lead5': (125, 680, 1125, 845), 'Lead6': (1125, 680, 2125, 845),
                     'Lead7': (125, 845, 1125, 1000), 'Lead8': (1125, 845, 2125, 1000),
                     'Lead9': (125, 1000, 1125, 1150), 'Lead10': (1125, 1000, 2125, 1150),
                    'Lead11': (125, 1150, 1125, 1300), 'Lead12': (1125, 1150, 2125, 1300),
                     'Lead13': (125, 1300, 2125, 1490)},

                    {'Lead1': (125, 310, 625, 600), 'Lead2': (640, 310, 1125, 600), 'Lead3': (1140, 310, 1625, 600), 'Lead4': (1640, 310, 2125, 600),
                     'Lead5': (125, 605, 625, 900), 'Lead6': (640, 605, 1125, 900), 'Lead7': (1140, 605, 1625, 900), 'Lead8': (1640, 605, 2125, 900),
                     'Lead9': (125, 905, 625, 1200), 'Lead10': (640, 905, 1125, 1200), 'Lead11': (1140, 905, 1625, 1200), 'Lead12': (1640, 905, 2125, 1200),
                     'Lead13': (125, 1205, 2125, 1495)}],

             'Covid': [{'Lead1': (100, 110, 300, 240), 'Lead2': (300, 110, 500, 240), 'Lead3': (500, 110, 700, 240), 'Lead4': (700, 110, 900, 240),
                        'Lead5': (100, 241, 300, 350), 'Lead6': (300, 241, 500, 350), 'Lead7': (500, 241, 700, 350), 'Lead8': (700, 241, 900, 350),
                        'Lead9': (100, 351, 300, 450), 'Lead10': (300, 351, 500, 450), 'Lead11': (500, 351, 700, 450), 'Lead12': (700, 351, 900, 450),
                        'Lead13': (100, 451, 900, 545)},

                       {'Lead1': (125, 310, 625, 600), 'Lead2': (640, 310, 1125, 600), 'Lead3': (1140, 310, 1625, 600), 'Lead4': (1640, 310, 2125, 600),
                        'Lead5': (125, 605, 625, 900), 'Lead6': (640, 605, 1125, 900), 'Lead7': (1140, 605, 1625, 900), 'Lead8': (1640, 605, 2125, 900),
                        'Lead9': (125, 905, 625, 1200), 'Lead10': (640, 905, 1125, 1200), 'Lead11': (1140, 905, 1625, 1200), 'Lead12': (1640, 905, 2125, 1200),
                        'Lead13': (125, 1205, 2125, 1495)}],

             'PMI': [{'Lead1': (125, 310, 625, 600), 'Lead2': (640, 310, 1125, 600), 'Lead3': (1140, 310, 1625, 600), 'Lead4': (1640, 310, 2125, 600),
                      'Lead5': (125, 605, 625, 900), 'Lead6': (640, 605, 1125, 900), 'Lead7': (1140, 605, 1625, 900), 'Lead8': (1640, 605, 2125, 900),
                      'Lead9': (125, 905, 625, 1200), 'Lead10': (640, 905, 1125, 1200), 'Lead11': (1140, 905, 1625, 1200), 'Lead12': (1640, 905, 2125, 1200),
                      'Lead13': (125, 1205, 2125, 1495)},

                     {'Lead1': (125, 410, 1125, 590), 'Lead2': (1125, 410, 2125, 590),
                      'Lead3': (125, 600, 1125, 815), 'Lead4': (1125, 600, 2125, 815),
                      'Lead5': (125, 815, 1125, 1035), 'Lead6': (1125, 815, 2125, 1035),
                      'Lead7': (125, 1035, 1125, 1235), 'Lead8': (1125, 1035, 2125, 1235),
                      'Lead9': (125, 1235, 1125, 1350), 'Lead10': (1125, 1235, 2125, 1350),
                      'Lead11': (125, 1350, 1125, 1490), 'Lead12': (1125, 1350, 2125, 1490),
                      },

                     {'Lead1': (125, 290, 1125, 490), 'Lead2': (1125, 290, 2125, 490),
                      'Lead3': (125, 490, 1125, 650), 'Lead4': (1125, 490, 2125, 650),
                      'Lead5': (125, 650, 1125, 810), 'Lead6': (1125, 650, 2125, 810),
                      'Lead7': (125, 810, 1125, 1000), 'Lead8': (1125, 810, 2125, 1000),
                      'Lead9': (125, 1000, 1125, 1190), 'Lead10': (1125, 1000, 2125, 1190),
                      'Lead11': (125, 1190, 1125, 1315), 'Lead12': (1125, 1190, 2125, 1315),
                      'Lead13': (125, 1315, 2125, 1550)
                      },

                     {'Lead1': (125, 310, 625, 600), 'Lead2': (640, 310, 1125, 600), 'Lead3': (1140, 310, 1625, 600), 'Lead4': (1640, 310, 2125, 600),
                      'Lead5': (125, 605, 625, 900), 'Lead6': (640, 605, 1125, 900), 'Lead7': (1140, 605, 1625, 900), 'Lead8': (1640, 605, 2125, 900),
                      'Lead9': (125, 905, 625, 1200), 'Lead10': (640, 905, 1125, 1200), 'Lead11': (1140, 905, 1625, 1200), 'Lead12': (1640, 905, 2125, 1200),
                      'Lead13': (125, 1205, 2125, 1495)}]
             }


In [5]:
dirList=['Normal','HB','MI','Covid','PMI']
# dirList=['ECGImagesofMyocardialInfarctionPatients']

In [12]:
def processor(dirs,path2,im,of,crop_dict,numLeads=13,threshold_level=50):

    for i in range(1,numLeads+1):

        # crop with the given coordinates
        imCrop = im.crop(box=crop_dict['Lead'+str(i)])

        # rename the cropped image
        target = of+'_cropped_lead_'+str(i)

        # make imcrop a numpy array
        imCrop = np.array(imCrop)

        # # save the cropped image
        # imCrop.save(os.path.join(path2,target+'.png'), "PNG", quality=100)

        # # prepare the cropped image for the next step
        # img = cv2.imread(os.path.join(path2,target+'.png'))

        # convert the image to grayscale with cv2
        

        # convert the grayscale image to binary image
        coords = np.column_stack(np.where(gray < threshold_level))

        # save the coordinates of the binary image
        coords[:,[0,1]]=coords[:,[1,0]]

        csvFolder = os.path.join(targetFolder,dirs)
        if not os.path.exists(csvFolder):
            os.makedirs(csvFolder)
        np.savetxt(os.path.join(csvFolder,target+'.csv'),coords)

        # save the grayscale image
        cv2.imwrite(os.path.join(path2,target+'.png'),gray)


In [7]:
def crop2csv(dirs,path,path2,dirList,superDict,pattern='.jpg'):

    count = 0
    for item in os.listdir(path):

        if os.path.isfile(os.path.join(path,item)):

            of, oe = os.path.splitext(item)
            
            if oe == pattern:
                
                im = Image.open(os.path.join(path,item))

                if dirs == dirList[1]:

                    if int(of[3:]) <= 399:
                        processor(dirs,path2,im,of,superDict[dirs][0])

                    elif (int(of[3:]) >= 400) and (int(of[3:]) <= 483):
                        processor(dirs,path2,im,of,superDict[dirs][1])

                    elif (int(of[3:]) >= 549):
                        processor(dirs,path2,im,of,superDict[dirs][2])
                
                elif dirs == dirList[2]:

                    if int(of[3:]) <= 37:
                        processor(dirs,path2,im,of,superDict[dirs][0])

                    elif (int(of[3:]) >= 38) and (int(of[3:]) <= 77):
                        processor(dirs,path2,im,of,superDict[dirs][1])

                    else:
                        processor(dirs,path2,im,of,superDict[dirs][2])

                elif dirs == dirList[3]:

                    if of[0:5] == 'COVID':
                        processor(dirs,path2,im,of,superDict[dirs][1])

                    else:
                        processor(dirs,path2,im,of,superDict[dirs][0])

                elif dirs == dirList[4]:

                    if int(of[4:]) < 113:
                        processor(dirs,path2,im,of,superDict[dirs][0])

                    elif (int(of[4:]) >= 113) and (int(of[4:]) <= 161) :
                        processor(dirs,path2,im,of,superDict[dirs][1],numLeads=12)
                        
                    elif (int(of[4:]) >= 162) and (int(of[4:]) <= 203) :
                        processor(dirs,path2,im,of,superDict[dirs][2])

                    else:
                        processor(dirs,path2,im,of,superDict[dirs][3])
                else:
                    processor(dirs,path2,im,of,superDict[dirs])
                    
            count = count+1
    return count

In [13]:
## Cropping
targetFolder = 'CSV_data'

print("hello")

if not os.path.exists(targetFolder):
    os.makedirs(targetFolder)

t0 = time.time()

for dirs in os.listdir(pathroot):

    t = time.time()
    print('Processing {0} folder ...'.format(dirs))

    if not os.path.isfile(dirs):

        path = os.path.join(pathroot,dirs)
        path2 = os.path.join(path,"Cropped_Images")

        if not os.path.exists(path2):
            os.makedirs(path2)

        fileCount = crop2csv(dirs,path,path2,dirList,superDict) 

    t = time.time()-t
    print('{0} files processed in this folder in {1} sec...\n'.format(fileCount,round(t)))
    
t0 = time.time()-t0

hello
Processing Covid folder ...
250 files processed in this folder in 20 sec...

Processing HB folder ...
714 files processed in this folder in 280 sec...

Processing MI folder ...
313 files processed in this folder in 98 sec...

Processing Normal folder ...
1143 files processed in this folder in 313 sec...

Processing PMI folder ...
326 files processed in this folder in 106 sec...

